# 📓 Please Upvote the notebook 📓

In [ ]:
import os
import pandas
import numpy as np
import matplotlib.pyplot as plt
import cv2

from fastai.vision.all import *

In [ ]:
!pip install fastai timm nbdev -q --upgrade

In [ ]:
import timm

In [ ]:
path = '/kaggle/input/landmark-recognition-2021/'

In [ ]:
train_data = pd.read_csv(path+'train.csv')
samp_subm = pd.read_csv(path+'sample_submission.csv')

In [ ]:
train_data.head(5)


In [ ]:
print(len(train_data))

In [ ]:
samp_subm.head(5)

#### Now we will get the path to all our images using a fast.ai function

In [ ]:
train_img_files = get_image_files(f'{path}train') #fastai function that helps us grab all the image files (recursively) in one folder.
len(train_img_files)

In [ ]:
train_img_files[0] #now we have the path of each image

In [ ]:
train_img_files[0].name

#### Now let us see the image and the shape for few

In [ ]:
for i in range(5):
    img = PILImage.create(train_img_files[i])
    img.show()
    print(img.shape)

#### So we can see that the shape for the images are all diffrent

In [ ]:
bs = 8 #batch size

#### We would pass in the train_data later so for that we would collect X and Y so we need to make functions for it

In [ ]:
get_x = lambda x: path/'train'/f'{x[0][0]}'/f'{x[0][1]}'/f'{x[0][2]}'/f'{x[0]}.jpg'

In [ ]:
get_y = lambda x: x[1]

In [ ]:
item_tfms = Resize(size = 460)
batch_tfms = [*aug_transforms(size = 224, min_scale = 0.35), Normalize.from_stats(*imagenet_stats)]

#### Creating a Datablock

In [ ]:
landmark_dataset = DataBlock(blocks = (ImageBlock, CategoryBlock),
                             get_x = get_x,
                             splitter = RandomSplitter(),
                             get_y = get_y,
                             item_tfms = item_tfms ,
                             batch_tfms = batch_tfms)

In [ ]:
dls = landmark_dataset.dataloaders(train_data, bs = bs)

In [ ]:
dls.show_batch(max_n = 8)

#### Now we will train the model

In [ ]:
learn = cnn_learner(dls, resnet34, metrics=accuracy, pretrained=False)

In [ ]:
learn.fit_one_cycle(5, 5e-3)

#### Now we will select a good learning rate

In [ ]:
learn.lr_find()

#### Now we will save our model and do prediction in the inference notebook

In [ ]:
learn.export()
path = Path()
path.ls(file_exts='.pkl')